# Cerebellar Model Articulation Controller (CMAC)


### Formulation

This is a proposed computing model based on the principles of the brain's architecture. This is similar in inspiration to multilayer perceptrons (aka. MLPs, neural networks).

In a similar way to MLPs, the model can be defined as [1,2]:

$$f : S \rightarrow A$$
$$g : A \rightarrow P$$

and $$y = \sum A^{*}_i w_i$$

Where $A$ are association cells (i.e. memory cells), $P$ are response output vectors, $S$ are the sensory input vectors.

Here we essentially say, the output is the sum of all the memory cells. Where we store weights in a memory table. Learn these by filling in values of the table cells (initially set to 0). By repeatedly trying to solve a task, we can adjust these weights. And learning is controlled through supervised learning (compare real vs. predicted).

BUT WE NEED TO GENERALIZE! This cell architecture is prone to non-generality. So we need a mapping function that determines a **set** of cells ($A^*$) for a given input vector ($X=(x_1,x_2,...,x_n)$). This helps generalize the cells to inputs. And thus the sum becomes: (1) map $X_1 \rightarrow A_1^{*}$ and $X_2 \rightarrow A_2^{*}$ then comput the sums of $A_1^*,A_2^*$ respectively.

In designing the mapping function, the increase of the size of $A^{*}$ will make the model more general.

Since only a small fraction of the table is used, use a hashing function.

To learn it, compute the error and add: $\frac{(y-\hat{y})}{|A^{*}|}$ to each cell in $A^{*}$ where $\hat{y}$ is the prediction ($\sum A^{*}$) and $y$ is the ground truth. This is the learning method proposed in [3].

This can be generally viwed as a neural network in some sense and then formulated as a table lookup:

![cmac](images/cmac_formulation.png)
TODO: caption this somehow as Figure 1. Note: this image is taken from: [4].

### Relation to the human brain

Looking at Figure 1, we can correlate each section loosely to the human brain. However, these are loose correlations and do not accurately model the cerebellum. More importantly the model is fast as it can be viwed as a lookup table.

#### Feature Detecting Neurons
The first layer is related to sensory feature-detecting neurons.

#### Granule cells
The second layer in Figure 1 corresponds to granule cells.

#### Purkinje cells
The third layer in Figure 1 corresponds to Purkinje cells.


### MLP vs CMAC

The difference between MLPs and CMAC [1] is that in MLPs $f$ is fully connected, while CMAC restricts the associations to neighbors. This is more in tune with Convolutional Nueral Networks in a sense (TODO cite).

### Differences from the human brain

There are suggestions that the cerebellum is capable of exclusive disjunction which is not in CMAC [1]. TODO more on this.

### Benefits of CMAC

"the restriction on mapping function effectively decreased the
chance of been trapped in a local minimum during learning
process" [1]

TODO: more on this

### Disadvantages

TODO

### Relation to TileCoding

In Tile Coding, you build 


### Relation to CNNs


### Our implementation of TileCoding in TensorFlow


## References

[1] Xing, Frank Z. "A Historical Review of Forty Years of Research on CMAC." arXiv preprint arXiv:1702.02277 (2017).

[2] http://www4.cs.umanitoba.ca/~jacky/Teaching/Courses/74.795-L02/2003/Lectures/cmac.pdf

[3] J.S. Albus (1975). "A New Approach to Manipulator Control: the Cerebellar Model Articulation Controller (CMAC)". In: Trans. ASME, Series G. Journal of Dynamic Systems, Measurement and Control, Vol. 97, pp. 220–233, 1975.

[4] Smith, Russell L. Intelligent motion control with an artificial cerebellum. Diss. ResearchSpace@ Auckland, 1998.